In [3]:
# импорт необходимых библиотек
import numpy as np
import pandas as pd

# импорт AWS SDK для питона 
import boto3 

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Для работы с облачным хранилищем AWS надо завести аккаунт: необходимо указать реквизиты банковской карты. 
С российскими картами это не получится. 

Можно ли обращаться к бакетам анонимно - тем более, если мой датасет публичный? 

Можно: https://stackoverflow.com/questions/34865927/can-i-use-boto3-anonymously

In [17]:
# модули для анонимных обращений
from botocore import UNSIGNED
from botocore.client import Config

In [35]:
# настройка клиента
s3 = boto3.client("s3", region_name='us-east-2', config=Config(signature_version=UNSIGNED))
s3_resource = boto3.resource("s3", region_name='us-east-2', config=Config(signature_version=UNSIGNED))

Обращаемся к бакету и смотрим, какие файлы в нем лежат. 

Сурс: https://sciwiki.fredhutch.org/compdemos/aws-s3/

In [55]:
bucketname = 'greenwichhr-covidjobimpacts'
response = s3.list_objects(Bucket=bucketname)
df = pd.DataFrame.from_dict(response['Contents'])
df

,Key,LastModified,ETag,Size,StorageClass,Owner
0,geography.csv.part_00000,2022-11-02 04:02:49+00:00,"""3a4ba65df206cb66c320df92a13889f4""",715891,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
1,geography_industry.csv.part_00000,2022-11-02 04:02:49+00:00,"""be850222838647fa165216ee48c4b8db-2""",17428149,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
2,ghr_data_specs_covid_public.pdf,2022-11-02 04:02:49+00:00,"""359201940fee9589e533ce8f0fd0f1bd""",105428,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
3,industry.csv.part_00000,2022-11-02 04:02:49+00:00,"""f34acb9c9594455236095951f8149cd2""",262557,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
4,industry_job_family.csv.part_00000,2022-11-02 04:02:49+00:00,"""0203f657d176f4aa77fa5f3e9df219d6""",7221882,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
5,job_family.csv.part_00000,2022-11-02 04:02:49+00:00,"""012a1f0cd17aef83adfbeff92ca30ecb""",357094,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
6,job_family_role.csv.part_00000,2022-11-02 04:02:49+00:00,"""1c3ae0cd8c6562d3268aae484ea29826-2""",31910685,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...
7,overall.csv.part_00000,2022-11-02 04:02:49+00:00,"""18955dbce2da45ec3298bac454baaee0""",10233,DEEP_ARCHIVE,{'ID': '062b3f99385d781b08a4a8c988c11872dfe3ea...


In [56]:
# сколько все это весит в гигабайтах? 
df.Size.sum() / (1024 * 1024)

55.324477195739746

In [60]:
# хотел спрятать огромное описание ошибок, но почему-то код ниже не помогает

#import sys
#sys.tracebacklimit = 0 

ERROR! Session/line number was not unique in database. History logging moved to new session 121


Попробуем что-нибудь скачать. 

Получаем ошибку.
> An error occurred (InvalidObjectState) when calling the GetObject operation: The operation is not valid for the object's storage class

Идем читать про storage classes. 

https://docs.aws.amazon.com/AmazonS3/latest/userguide/storage-class-intro.html#sc-compare
    
Смотрим в Deep Archive - класс всех объектов, которые лежат в бакете.
> You must first restore archived objects before you can access them. For information, see Restoring an archived object.

In [63]:
s3.download_file(bucketname, 'ghr_data_specs_covid_public.pdf', 'ayy_lmao')

InvalidObjectState: An error occurred (InvalidObjectState) when calling the GetObject operation: The operation is not valid for the object's storage class

Попытка восстановить объект на минимальный срок в один день - возьмем самый легкий. 

Заданные параметры возвращают распакованный из архива объект в тот же бакет.
Поскольку у меня нет прав для изменения бакета, получаем ошибку. 

А если бы у меня был аккаунт? Тогда мне бы пришлось заплатить денег, чтобы получить доступ к общественному датасету. Печаль. Надо выбрать что-нибудь другое для анализа. 
> When you restore an archived object, you are paying for both the archive and a copy that you restored temporarily.

In [61]:
s3.restore_object(
    Bucket=bucketname, 
    Key='overall.csv.part_00000', 
    RestoreRequest={
        'Days': 1,
        'GlacierJobParameters': {
            'Tier': 'Standard'
        },
    },
)

ClientError: An error occurred (AccessDenied) when calling the RestoreObject operation: Access Denied